# Startup Blueprint Generator Agent

## Problem Statement  
This system transforms raw startup ideas into complete, actionable business blueprints using **IBM Watsonx Granite model** and a **Retrieval-Augmented Generation (RAG)** approach.

For this eg : An app that lets students rent textbooks peer-to-peer.

It retrieves relevant data such as:
- Market research
- Funding options
- Competitor analysis
- Revenue models
- Government schemes
- Legal requirements

## Technology Used
- **IBM Watsonx API** (Granite-3-3-8b-instruct model)
- Python
- `.env` for secure credentials

This project was implemented using IBM Watsonx AI’s Granite Large Language Models (LLMs) accessed via IBM Cloud API.
The Python script connects to IBM’s hosted Granite model (granite-3-3-8b-instruct) using API credentials from an .env file, retrieves startup-related insights through prompt engineering, and writes a complete business plan to a text file.
This approach aligns with IBM Cloud Lite services usage because the computation and model hosting occur on IBM’s cloud infrastructure; the local machine (or Google Colab) only sends requests and receives responses.
No external non-IBM AI services were used in the final build.


## Step 1: Install Dependencies
We install the required Python packages to interact with IBM Watsonx.

In [ ]:
!pip install ibm-watsonx-ai python-dotenv

## Step 2: Set Up Environment Variables
Replace placeholders in the `.env` file with your actual IBM Cloud API credentials before running.


# NOTE for reviewers:
The actual IBM_API_KEY has been replaced with "YOUR_API_KEY" in this public notebook to avoid exposing sensitive credentials.
To run this notebook, replace "YOUR_API_KEY" in the .env file with a valid key from your IBM Cloud account.

In [ ]:
%%writefile .env
IBM_API_KEY= "YOUR_API_KEY"
PROJECT_ID=f4fe3f58-8d9d-43ba-b45c-3b6bcabca8c8
REGION=us-south
MODEL_ID=ibm/granite-3-3-8b-instruct
URL=https://us-south.ml.cloud.ibm.com

Overwriting .env


## Step 3: Import Libraries and Load Environment Variables

In [ ]:
import os
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.credentials import Credentials
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("IBM_API_KEY")
REGION = os.getenv("REGION")
MODEL_ID = os.getenv("MODEL_ID")
PROJECT_ID = os.getenv("PROJECT_ID")

creds = Credentials(
    api_key=API_KEY,
    url=f"https://{REGION}.ml.cloud.ibm.com"
)

model = ModelInference(
    model_id=MODEL_ID,
    credentials=creds,
    project_id=PROJECT_ID
)

## Step 4: Define Startup Idea and Sections
We will collect the idea from the user and define the sections for the business plan.

In [ ]:
idea = input("Enter your startup idea: ")

sections = [
    "Market Opportunity",
    "Business Model",
    "Estimated Budget",
    "Go-to-Market Strategy",
    "Potential Competitors",
    "Revenue Streams",
    "Risks and Challenges",
    "Key Performance Indicators (KPIs)"
]

with open("output.txt", "w") as f:
    f.write(f"📦 Startup Blueprint for: {idea}\n\n")


Enter your startup idea: An app that lets students rent textbooks peer-to-peer.


## Step 5: Generate Each Section Using IBM Granite

In [ ]:
from time import sleep

def safe_generate(prompt, max_tokens=1500):
    """Generate text with optional max_tokens if supported."""
    try:
        return model.generate_text(prompt, max_new_tokens=max_tokens, temperature=0.7)
    except TypeError:
        return model.generate_text(prompt)

for section in sections:
    base_prompt = f"Generate the '{section}' section of a startup business plan for this idea: {idea}. Be detailed, structured, and complete."
    try:
        # 1. First attempt
        response = safe_generate(base_prompt)
        if not isinstance(response, str):
            response = str(response)
        full_text = response.strip()

        # 2. Try continuation if output looks incomplete
        for _ in range(4):  # up to 4 extra chunks
            if len(full_text.split()) < 120 or full_text[-1] not in ".!?\"'":
                cont_prompt = (
                    f"You are writing the '{section}' section of the startup business plan "
                    f"for the idea: {idea}.\n\nHere is what you have so far:\n{full_text}\n\n"
                    "Continue directly from the last sentence, without repeating anything, "
                    "until the section is fully complete. Stay 100% on topic."
                )
                cont_chunk = safe_generate(cont_prompt, max_tokens=1000).strip()
                if not cont_chunk or cont_chunk in full_text:
                    break
                full_text += " " + cont_chunk
                sleep(0.3)
            else:
                break

        # 3. Save
        print(f"\n\n=== {section} ===\n{full_text}\n")
        with open("output.txt", "a") as f:
            f.write(f"\n\n{section}:\n{full_text}\n")
        print(f"✅ {section} written.")

    except Exception as e:
        print(f"❌ Error in {section}: {e}")


=== Market Opportunity ===
**Market Opportunity**

The global textbook market is valued in the billions annually, with students often facing high costs for purchasing new books. A peer-to-peer textbook rental platform can significantly reduce expenses for students while also promoting sustainability by extending the life of books. Universities and colleges present a concentrated and easily targetable user base, ensuring a strong initial adoption potential. With increasing internet penetration and the normalization of app-based services, the timing is ideal for launching such a solution.
✅ Market Opportunity written.

=== Business Model ===
**Business Model**

1. **Value Proposition** – Affordable and eco-friendly access to textbooks for students through peer-to-peer rentals.
2. **Customer Segments** – Primary: College and university students. Secondary: Independent learners and academic institutions.
3. **Channels** – Mobile app, campus ambassador programs, and partnerships with stude